In [1]:
import pandas as pd

# df = pd.read_csv('youtube_links.csv')

# urls = df['link'].tolist()[113:152]

urls = [
    'https://www.youtube.com/watch?v=rvLYSt5K2Ws',
    'https://www.youtube.com/watch?v=j9B0N5iq9d4',
    'https://www.youtube.com/watch?v=CIKwkCOTEYk',
    'https://www.youtube.com/watch?v=01DhQzgtmpw',
    'https://www.youtube.com/watch?v=TKpv-P09Z2o'
]

In [3]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from lxml import etree
from selenium.webdriver.common.by import By

# 操作已经打开的浏览器
options = Options()
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(10)

data = {
    'title': [],
    'count': [],
    'comment_text': [],
    'vote': [],
    'reply': [],
    'date': []
}


for url in urls:
    # url = f'https://www.youtube.com{url}'
    print(url)
    driver.get(url)
    time.sleep(2)
    driver.execute_script("var q=document.documentElement.scrollTop=300")
    comment_count_last = 0
    time.sleep(2)
    page = driver.page_source
    tree = etree.HTML(page)
    title = tree.xpath('.//yt-formatted-string[@class="style-scope ytd-watch-metadata"]/text()')[0].strip()
    # if 'CG' not in title and '不夜长安' not in title:
    #     continue

    for i in range(1, 1000):
        # 滚轮
        driver.execute_script("var q=document.documentElement.scrollTop=1000000000")
        time.sleep(2)

        if i % 5 == 0:
            # 获取网页源代码
            page = driver.page_source
            tree = etree.HTML(page)
            comment_list = tree.xpath(
                './/div[@id="contents"]/ytd-comment-thread-renderer[@class="style-scope ytd-item-section-renderer"]')
            comment_count = len(comment_list)
            if comment_count == comment_count_last:
                print(len(comment_list))
                break
            else:
                comment_count_last = comment_count
    print('轮动结束')


    reply_button = driver.find_elements(By.XPATH, './/button[contains(@aria-label, "repl")]')[::2]
    for button in reply_button:
        time.sleep(2)
        driver.execute_script("arguments[0].scrollIntoView();", button) 
        time.sleep(0.5)
        try:
            driver.execute_script("arguments[0].click();", button)

        except:
            pass
    print('点击结束')



    time.sleep(4)
    page = driver.page_source
    tree = etree.HTML(page)
    print('开始解析')
    try:
        count = tree.xpath('.//h2[@id="count"]/yt-formatted-string/span[@dir="auto"]/text()')[0].strip()
    except:
        count = 0
    title = tree.xpath('.//yt-formatted-string[@class="style-scope ytd-watch-metadata"]/text()')[0].strip()

    comment_list = tree.xpath('.//div[@id="contents"]/ytd-comment-thread-renderer[@class="style-scope ytd-item-section-renderer"]')
    reply_list = tree.xpath('.//div[@id="contents"]//ytd-comment-renderer[@class="style-scope ytd-comment-replies-renderer"]//div[@id="main"]')
    print(len(comment_list), len(reply_list))
    for comment in comment_list:
        text = comment.xpath('.//yt-formatted-string[@id="content-text"]//text()')
        text = [t for t in text if t.strip() != '']
        comment_text = '\n'.join(text).strip()
        try:
            vote = comment.xpath('.//span[@id="vote-count-middle"]/text()')[0].strip()
        except:
            vote = 0
        try:
            reply = comment.xpath('.//button[@class="yt-spec-button-shape-next yt-spec-button-shape-next--text yt-spec-button-shape-next--call-to-action yt-spec-button-shape-next--size-m yt-spec-button-shape-next--icon-leading yt-spec-button-shape-next--align-by-text"]/@aria-label')[0].strip()
        except:
            reply = 0
        date = comment.xpath('.//a[@class="yt-simple-endpoint style-scope yt-formatted-string"]/text()')[0].strip()

        data['comment_text'].append(comment_text)
        data['vote'].append(vote)
        data['reply'].append(reply)
        data['date'].append(date)
        data['count'].append(count)
        data['title'].append(title)
    
    for rep in reply_list:
        text = rep.xpath('.//yt-formatted-string[@id="content-text"]//text()')
        text = [t for t in text if t.strip() != '']
        comment_text = '\n'.join(text).strip()
        try:
            vote = rep.xpath('.//span[@id="vote-count-left"]/text()')[0].strip()
        except:
            vote = 0
        try:
            reply = rep.xpath('.//div[@id="reply-button"]/text()')[0].strip()
        except:
            reply = 0
        date = rep.xpath('.//a[@class="yt-simple-endpoint style-scope yt-formatted-string"]/text()')[0].strip()

        data['comment_text'].append(comment_text)
        data['vote'].append(vote)
        data['reply'].append(reply)
        data['date'].append(date)
        data['count'].append(count)
        data['title'].append(title)


    df = pd.DataFrame(data)
    df.to_csv('youtube 账号下的.csv', index=False, encoding='utf-8-sig')

https://www.youtube.com/watch?v=rvLYSt5K2Ws
57
轮动结束
点击结束
开始解析
57 0
https://www.youtube.com/watch?v=j9B0N5iq9d4
424
轮动结束
点击结束
开始解析
424 145
https://www.youtube.com/watch?v=CIKwkCOTEYk
141
轮动结束
点击结束
开始解析
141 51
https://www.youtube.com/watch?v=01DhQzgtmpw
24
轮动结束
点击结束
开始解析
24 0
https://www.youtube.com/watch?v=TKpv-P09Z2o
205
轮动结束
点击结束
开始解析
205 68
